# <span style="color:Blue"> T4b: Advanced EELS Processing </span>

# Table of Contents<a name="toc"/>
**[1. Signal limited data](#start)**
* [1a. Importing data](#import)
* [1b. Calibrating the energy axis](#cal)
* [1c. Log vs. non-log fitting](#log)
* [1d. Elemental mapping](#map)

**[2. MCR](#mcr)**
* [2a. Introduction to MCR ](#mcr1)
* [2b. Generating guess spectra](#mcr2)
* [2c. Identifying principal components of SI](#mcr3)

**[3. Additional Tools (optional)](#tools)**
* [3a. Zero Loss Peak Shifting](#shift)
* [3b. Additional fitting parameters](#fit)

<span style="color:orange">**Note: At the end of each section, please check with your group's TA and ask any questions that you might have about what you've just gone through. Take this opportunity to discuss the section with others in your group! Also, feel free to ask questions at any point in this tutorial. Make sure you have a good grasp on what it is you're doing in each section before moving on!**</span>

In this notebook we will explore some more advanced EELS processing techniques such as background subtraction on low count data and Multivariate Curve Resolution (MCR).

# <span style="color:Purple"> 1. Signal limited data </span>  <a name="start"/> 

## <span style="color:Purple"> 1a. Importing data </span> <a name="import"/> 

Begin this notebook by importing all of the necessary packages for this notebook.

In [ ]:
## Imports
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs

import eels
import MCR_Functions as mf

In [ ]:
# file path for directory with data files
path = "../Data/T4b/" 

# data file name
SIdm = "Example 1 EELS Spectrum Image.dm4"   

## Load the SI (spectral image), calibrated energy axis, and the metadata parameters
rawE, rawSI, pxscale, disp, params = eels.specload(path+SIdm,type='1')

xdim,ydim,edim = np.shape(rawSI)
sumspec = np.sum(rawSI,axis=(0,1))

This SI data set is an atomic resolution EELS map on an interface that we will work to identify throughout this first section. This data set was also taken at **cryogenic temperatures (~90 K)**, which requires very short acquisition times. As you will see later in this notebook, anything that limits the overal signal in your EELS data (dwell time, electron dose, weak scattering, etc.) will present some challenges for us when processing the data.

Before we create our edge maps, let's study the spectra across the interface. To do this, we will use the ***`eels.edge_check`*** GUI. This will allow sum over discrete regions of the SI to gain an understanding of how the distribution of elements changes.

Try to identify the elements on either side of this interface. In your browser, go to [EELS.info](https://eels.info/atlas) or [WEELS.net](http://muller.research.engineering.cornell.edu/sites/WEELS/). These websites contain a **database of EELS edges** corresponding to each of the elements on the periodic table. You can either pick a specific element to see it's edges or look up a given energy below the periodic table to find all elements that have an edge near that energy.

In [ ]:
eels.edge_check(rawSI,rawE)

***How does the spectrum change across the interface? What edges are on either side of the interface and what elements do these edges correspond to?***

***Are there any common edges? Do those edges have a consistent shape on either side of the interface?***

<span style="color:orange">**Please confirm with your TA that you have properly identified the elements in this data set before moving on.**</span>

## <span style="color:Purple"> 1b. Calibrating the energy axis </span> <a name="cal"/> 

While we already have an energy array corresponding to spectra in our SI, it is often neccessary to shift this energy axis to match a certain element's *expected* energy loss. This should be done on a stable, sharp edge in the SI, where the EELS edge does not shift depending on the element's valence structure.

In this example, we have two sharp edges around 830 eV. These are the La M5 and M4 edges. These are stable, sharp edges that we can use to calibrate our energy axis.

We will plot this edge below. ***How much should we shift the energy axis by so that the M5 edge peak lines up with the energy provided by the EELS data base?*** Use [EELS.info](https://eels.info/atlas) to check the expected location of these two edges.

In [ ]:
shift = ?

eels.easyplot("Energy Correction")
plt.plot(rawE + shift, sumspec, color = 'b')
plt.xlim(820,860)

Once you are happy with the shift applied to the energy axis, run the cell below to shift the energy axis.

In [ ]:
## Warning: make sure to only run this cell once. Every time this cell is ran, it will apply another shift to the energy axis.

rawE += shift

## <span style="color:Purple"> 1c. Log vs. non-log fitting </span>  <a name="log"/> 

Let's start by attempting to background subtract the La M5 and M4 edges. This EELS data set was acuqired with lower total signal than the first data set we looked at. This data set was also, like the first data set, collected by a **direct electron detector** (K2 Summit). This means the counts we see here correspond to individual electron detection events. Lets look at the spectrum generated at a single pixel in this data set. Focus on the values for the counts and see if you can tell why the faster log fitting method we used in the first method might not work in this data set.

In [ ]:
## Pick a pixel position (ideally where you've already identified Lanthanum to be!)

xpos,ypos = ??,??

eels.easyplot('Single pixel counts')
plt.plot(rawE,rawSI[xpos,ypos,:])
plt.xlim([800,900])

**Do you see a problem with using a log transform on this SI?**

Let's now use some of the functions shown earlier to determine the proper fitting conditions for these La M4,5 edges.

In [ ]:
eels.fit_check(rawSI,rawE)

Now lets try a background subtraction using both a **log transform and a non-log transform**, which uses a non-linear least squares fitting. 

Note: the non-log fitting will be much slower than the log fitting, but this is sometimes neccessary in low count data.

In [ ]:
## Define edge parameter

La = [??,??,??,??,'La-M4,5']

edge = La

## Log fitting
bsub_pl_log = eels.bgsub_SI(rawSI, rawE, edge,log=True)
## Non-log fitting
bsub_pl = eels.bgsub_SI(rawSI, rawE, edge,log=False) 

In [ ]:
## Compare these background subtractions

## Define edge interms of channel number for conveniece
edge_ch = []
for element in edge[:-1]:
    edge_ch.append(eels.eVtoCh(element,rawE))
    
## For plotting convience
emin = edge[2] - 20.
emax = edge[3] + 10.
chmin = eels.eVtoCh(emin, rawE)
chmax = eels.eVtoCh(emax, rawE)

fig,(ax1,ax2)=plt.subplots(1,2,figsize=(8,4))

ax1.plot(rawE[chmin:chmax], sum(sum(bsub_pl_log))[chmin:chmax],label='Log')
ax1.plot(rawE[chmin:chmax], sum(sum(bsub_pl))[chmin:chmax],label='Non-Log')
ax1.set_xlim(emin, emax)
ax1.axhline(0, color='k', linestyle='--')
ax1.legend()
ax1.set_xlim()
ax1.set_title('Log vs. Non-Log comparison')

ax2.set_title(' (Post-edge)')
ax2.plot(rawE[chmax:chmax+500], sum(sum(bsub_pl_log))[chmax:chmax+500],label='Log')
ax2.plot(rawE[chmax:chmax+500], sum(sum(bsub_pl))[chmax:chmax+500],label='Non-Log')
ax2.set_xlim(emax, emax+500*disp)
ax2.axhline(0, color='k', linestyle='--')
ax2.legend()
ax2.set_xlim()

edgemap = eels.edgemap(bsub_pl, rawE, edge)
edgemap_log = eels.edgemap(bsub_pl_log, rawE, edge)

fig,(ax1,ax2) = plt.subplots(1,2)
ax1.imshow(edgemap_log,cmap='gray')
ax1.set_title('Log ')
ax2.imshow(edgemap,cmap='gray')
ax2.set_title('Non-log ')

**What are the signs here that the log fitting has failed?**

In the first notebook, we went over how to indentify a quality fit. The first being the pre-edge fit window flat and centered at zero, and the second being the power law shape of the post edge. 

**Why do you think the log fit failed?**

## <span style="color:Purple"> 1d. Elemental mapping </span> <a name="map"/> 

Now that you are equiped with all of the neccessary tools for EELS analysis, let's work to create a compositional map of three elements in this SI, Lanthanum, Titanium and Manganese. 

Remember the functions you've learned along the way!

Key Points:
- **`eels.fit_check(SI,rawE)`** : This function helps determine the proper fitting window for your edge.
- **`edge = [fit start, fit end, int start, int end,'edge label']`**  : Don't forget how to save your edge parameters.
- **`eels.bgsub_SI(SI, rawE, edge)`**  : Use this function to perform a background subtraction. In this example we will want to use ***`log=False`*** and ***`gfwhm = num`*** to improve fitting quality.
- **`eels.edge_check(bsub,rawE,edge)`** : This GUI will help you check the quality of your background subtraction.
- **`eels.edgemap(bsub, rawE, edge)`**  : This will create your elemental maps.
- **`np.savez(path + title, bsub = bsub, energy = rawE, edge = edge)`** : Make sure to save your background subtracted data!

Start by picking your parameters using ***`eels.fit_check`***.

In [ ]:
eels.fit_check(rawSI,rawE)

Perform each background subtraction one at a time and save as an npz file.

In [ ]:
## Assign params to a variable by edge
## Return to this cell to perform a background subtraction for each edge.

Ti = [??,??,??,??,'Ti-L2,3']
Mn = [??,??,??,??,'Mn-L2,3']
La = [??,??,??,??,'La-M4,5']

edge = La

gfwhm = 15

## Define titles used for saving convenience later on
title = edge[4]+'_fit'+str(int(edge[0]))+'-'+str(int(edge[1]))+'_int'+str(int(edge[2]))+'-'+str(int(edge[3]))+ '_lba_'+str(gfwhm)

bsub = eels.bgsub_SI(rawSI, rawE, edge, log= False, gfwhm=gfwhm)

In [ ]:
## Ensure that you have a flat background centered at zero. 

eels.edge_check(bsub,rawE,edge)

<span style="color:orange">**If you are having a hard time getting a good background subtraction or are uncertain about the quality of the background subtraction, ask the TA for help!**</span>

In [ ]:
## Don't forget to save your data.
np.savez(title, 
        bsub = bsub,
        energy = rawE, 
        edge = edge)

Now that we have our background subtracted data, we can create a compositional map similar to what we did in the first example.

In [ ]:
## Once you have performed satisfactory background subtractions, load in your npz files here:

Ti_bsub,Ti_energy,Ti_edge = eels.load_bsub('???')
Mn_bsub,Mn_energy,Mn_edge = eels.load_bsub('???')
La_bsub,La_energy,La_edge = eels.load_bsub('???')

In [ ]:
## Lets first take these background subtracted edges and create edgemaps as shown earlier

Ti_edgemap = eels.edgemap(Ti_bsub,Ti_energy,Ti_edge)
Mn_edgemap = eels.edgemap(Mn_bsub,Mn_energy,Mn_edge)
La_edgemap = eels.edgemap(La_bsub,La_energy,La_edge)

fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(9,3))
ax1.imshow(Ti_edgemap,cmap='gray')
ax1.set_title('Ti-L2,3')
ax2.imshow(Mn_edgemap,cmap='gray')
ax2.set_title('Mn-L2,3')
ax3.imshow(La_edgemap,cmap='gray')
ax3.set_title('La-M4,5')

rgbmap = eels.easyrgb([La_edgemap,Ti_edgemap,Mn_edgemap],['r','b','g'])     

fig,ax=plt.subplots()
ax.set_title('La: Red , Mn : Green , Ti : Blue') # change this based on the colors you choose above
ax.imshow(rgbmap)

This SI is a map of an **LSMO (La$_{1-x}$Sr$_x$Mn O$_3$)** - **STO (SrTiO$_3$)** interface. ***Does this correspond with your compositional map?***

We can also sum along the axis orthogonal to the interface to create line profile concentration plots of the elements in this SI.

In [ ]:
Mn_lineprofile = np.sum(Mn_edgemap, axis = 1)
Ti_lineprofile = np.sum(Ti_edgemap, axis = 1)
La_lineprofile = np.sum(La_edgemap, axis = 1)


eels.easyplot("line profile of concentration")
plt.plot(np.arange(len(Mn_lineprofile))*pxscale,Mn_lineprofile/np.max(Mn_lineprofile),'g',alpha=0.7,label ='Mn')
plt.plot(np.arange(len(Ti_lineprofile))*pxscale,Ti_lineprofile/np.max(Ti_lineprofile),'b',alpha=0.7,label ='Ti')
plt.plot(np.arange(len(La_lineprofile))*pxscale,La_lineprofile/np.max(La_lineprofile),'r',alpha=0.7,label ='La')
plt.xlabel('Distance (nm)')
plt.ylabel("Normalized Intensity (a.u.)")
plt.legend()

[Return to TOC](#toc)

# <span style="color:Purple"> 2. MCR </span> <a name="mcr"/> 

## <span style="color:Purple"> 2a. Introduction to MCR  </span> <a name="mcr1"/> 

Why do we go through the trouble of cooling down these materials to cryogenic temperatures for EELS analysis? There can be several motivations for this, but for a lot of quantum materials, we typically do this because we would like to study changes in the fine structure of the EELS edges.

You might have noticed in the first example that the Oxygen K edge changed shape on either side of the SI. This indicates a change in the density of states of conduction band electrons. We can identify any changes in the fine structure of the oxygen edge using MCR. 

MCR works by iteratively solving a least squares fit of a set of composition maps given a set of spectra, followed by set of spectra given a set of composition maps. 

The cell below is just a reminder of what that change in the Oxygen K edge looked like. We will be using a higher signal to noise data set taken at room temperature on a similar system for simplicity.

In [ ]:
O  = [505,523,525,572,'O-K']
bsub = eels.bgsub_SI(rawSI, rawE, O, log= False, gfwhm=gfwhm)
edgemap = eels.edgemap(bsub,rawE,O)
fig,ax = plt.subplots(1,2,figsize=(8,4))
ax[0].imshow(edgemap,cmap='gray')
box_1 = [[0.5,0.5,100.5,100.5,0.5],[0.5,ydim-0.5,ydim-0.5,0.5,0.5]]
box_2 = [[166.5,166.5,267.5,267.5,166.5],[0.5,ydim-0.5,ydim-0.5,0.5,0.5]]
ax[0].plot(box_1[1],box_1[0])
ax[0].plot(box_2[1],box_2[0])
ax[1].plot(rawE,np.sum(bsub[:100,:,:],axis=(0,1)))
ax[1].plot(rawE,np.sum(bsub[167:,:,:],axis=(0,1)))
ax[1].set_xlim(O[0],O[3]+20)
ax[1].set_ylim(-1e3,1e4)
ax[1].set_title('O-K edge Fine Structure')

## <span style="color:Purple"> 2b. Generating guess spectra  </span>  <a name="mcr2"/> 

The goal of MCR is to identify and map changes in the fine structure of an EELS edge. In order for this to work, we must provide a set of either guess compositions or spectra. In EELS, we typically provide a set of guess spectra by averaging the eels signal over distinct regions of the SI. 

We will start by loading in a pre-background subtracted Oxygen K edge data set. This SI was taken at the interface of **LSMO (La$_{1-x}$Sr$_x$Mn O$_3$)** - **STO (SrTiO$_3$)** interface. The **STO** is the lower half of this SI, and the **LSMO** is the top half. For this example, we will use MCR to identify and map the two unique fine structures of the Oxygen K edge in these materials.

Note: This SI is binned just after loading in this cell. This can help when performing MCR as it raises the SNR. We will show later in this example how to return from the binned SI to the unbinned SI.

In [ ]:
# file path for directory with data files
O_bsub, O_energy, O_edge  = eels.load_bsub(path+'O_Kedge_mcr_example.npz')

## load_check is a convient way to check our data file to ensure we are using the proper data set
mf.load_check(O_bsub, O_energy, O_edge)

## Optionally if we have a noisy data, we can bin our SI
## bin_factors decides binning factor in xdim, ydim and zdim

bin_factors=(2,2,1)

bin_SI=mf.bin_ndarray(O_bsub[:-1,:,:],bin_factors)

## more_params is a quick method of pulling out useful information
dimx,dimy,dimE, disp,edge_ch = mf.more_params(bin_SI, O_energy, O_edge)

Use ***`eels.edge_check`*** below to check the Oxygen K-edge on either side of the interface. Use the region 1 & region 2 options to help us generate guess spectra for MCR in the next cell. You can use the change in brightness to identify the edge of this interface.

In [ ]:
## Before defining guess spectra, it can be useful to scan our SI for disctinct O-K edges using eels.edge_check

eels.edge_check(bin_SI, O_energy, O_edge)

Now that we can see how the Oxygen edge changes in this system, we would like to isolate these two components of the oxygen edge in our SI and map where it changes in the SI.

In the cell below, we will define two regions where we will sum to generate two unique guess spectra for MCR. Becuase we want to map how the Oxygen edge changes in the two materials, the two guess spectra should be from a region soley on either side of the interface to isolate that component.

Use the cell above to help define the selection regions you will use to generate the guess spectra.

Important Note: Below, the cell defines a energy window. This window is the region of the background subtracted data that MCR actually works on. This is a very important step. We want to make sure MCR is only considering the edge of interest when determining the prinipal components of the SI.

In [ ]:
## Choosing guess spectra

########################################### User Inputs #####################################################################

## Define the coordinates for rectangular selection regions.
## [[x1,x2,y1,y2]] 

comp1=[??,??,??,??]
comp2=[??,??,??,??]

##############################################################################################################################

comps=np.stack((comp1,comp2))

## Define the energy window (in eV!) by selecting how far before and after the integration window you want to include in MCR.
pre_win=5.
post_win=5.
E_win=[edge_ch[2]-int(pre_win/disp),edge_ch[3]+int(post_win/disp)]

## Get the guess spectra 
guess_comp=mf.guess(bin_SI,comps,E_win)

## Get the edge map to display guess regions
edgemap = eels.edgemap(bin_SI,O_energy,O_edge)
## Get outlines of guess regions
outlines=mf.ROI_Outlines(bin_SI,comps)

## Show guesses
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(8,4))
ax1.imshow(edgemap,cmap='gray')
ax1.plot(outlines[0,1,:],outlines[0,0,:])
ax1.plot(outlines[1,1,:],outlines[1,0,:])
# ax1.plot(outlines[2,1,:],outlines[2,0,:])
ax1.set_title('Guess Regions')
ax2.plot(O_energy[E_win[0]:E_win[1]],guess_comp[0],label="guess 1")
ax2.plot(O_energy[E_win[0]:E_win[1]],guess_comp[1],label="guess 2")
# ax2.plot(O_energy[E_win[0]:E_win[1]],guess_comp[2],label="guess 3")
ax2.legend()
ax2.set_yticks([])
ax2.set_xlabel('Energy (eV)')
ax2.set_ylabel('Counts (a.u.)')
ax2.set_title('Guesses')

<span style="color:orange">**Before moving on, make sure you are confindent that you have properly chosen your guess spectra. Ask a TA if you are unsure of this step.**<span/>

## <span style="color:Purple"> 2c. Identifying principal components of SI  </span> <a name="mcr3"/> 

Now that we have our guess spectra, we are ready to run MCR! We do this with the function ***`mf.MCR`***. As you can see below, this function requires 5 inputs.
 - ***bin_SI*** : This is our background subtracted data cube. In this example, it is binned, but that is not necessary.
 - ***guess_comp*** : These are our guess spectra. The shape of this variable should be (n,m) where n is the number of guess spectra you will use, and m is the the size of the energy window you've chosen to run MCR on.
 - ***O_energy*** : This is the corresponding energy axis of the background subtracted data.
 - ***E_win*** : This is an array of two values that define the energy window that the data cube will be cropped to before running MCR. These two values are in channel number, not eV.
 - ***iterations*** : This is the number of iterations MCR will go through, where in each iteration it uses the components to generate concentration maps through least squares fitting, and then the concentration maps to regenerate the components. Be careful when using a high number of iterations, as MCR can start to orthogonalize the components and we loose their physical significance.

In [ ]:
iterations=2
scores,components=mf.MCR(bin_SI,guess_comp,O_energy,E_win,iterations=iterations)

Now that we have finished the MCR processing, we have our components, which are the principal spectra in the SI, and the scores, which tell us the concentration of each component in real space. Because we used the binned data in the MCR process, the score maps are also binned, but we can return these to the original dimensions of the raw data using ***`mf.full_scores`***. This function is essentially an extra half-step of MCR, where the unbinned data and components are used to find the scores using linear least squares.

In [ ]:
real_scores = mf.full_scores(O_bsub,scores,components,E_win)

#Show the results
fig,ax = plt.subplots(1,2,figsize = (10,6))
ax[0].imshow(real_scores[:,:,0],cmap='gray') 
ax[0].set_title('Score map 1')
ax[1].imshow(real_scores[:,:,1],cmap='gray') 
ax[1].set_title('Score map 2')

Now that we have our score maps and spectral components, we can get a clear picture of where the interface of **LSMO** and **STO** is in this SI and how the Oxygen bonding changes in the two materials.

In [ ]:
## RGB mapping of score maps to visualize spacial distribution of unique spectra
   
rgbmap = eels.easyrgb([real_scores[:,:,0],real_scores[:,:,1]],['c','m'])    

fig,(ax1,ax2)=plt.subplots(1,2,figsize=(9,4))
ax1.set_title('Score Map') # change this based on the colors you choose above
ax1.imshow(rgbmap)
ax2.set_title('MCR Components')
ax2.plot(O_energy[E_win[0]:E_win[1]],components[0,:],label = 'C1',color='c',alpha=0.7)
ax2.plot(O_energy[E_win[0]:E_win[1]],components[1,:],label = 'C2',color='m',alpha=0.7)
ax2.legend()
ax2.set_yticks([0])
ax2.set_ylabel('Counts (a.u.)')
ax2.set_xlabel('Energy (eV)')

Finally, we can also create line profiles as shown previously.

In [ ]:
lp1 = np.sum(real_scores[:,:,0], axis = 1)
lp2 = np.sum(real_scores[:,:,1], axis = 1)

eels.easyplot("line profile of concentration")
plt.plot(np.arange(len(lp1)),lp1/np.max(lp1),'c',alpha=0.7,label ='C1')
plt.plot(np.arange(len(lp2)),lp2/np.max(lp2),'m',alpha=0.7,label ='C2')
plt.xlabel('Distance (pix)')
plt.ylabel("Normalized Intensity (a.u.)")
plt.legend()

[Return to TOC](#toc)

#  <span style="color:Purple"> 3. Additional Tools (optional)  </span> <a name="tools"/> 

## <span style="color:Purple"> 3a. Zero Loss Peak Shifting  </span> <a name="shift"/> 

For low mag data, the electron beam can shift on the eels detector as we scan across the sample. This means that in our SI, the energy axis is no longer consistent for every pixel in the SI. When this happens, it can greatly effect your background subtraction, as you essentially have a drifting fit window. It also means the fine structure of an edge will be obscured. In this case, if you have a corresponding low loss data set with the zero loss peak, we can correct for this beam shift. This works best for Dual-EELS data sets when the low loss and core loss SI are collected simulatenously.

In [ ]:
## First, we load in both our CL_SI (core loss SI) and LL_SI (low loss SI)

# CL SI
CL_SIdm3 = "160kx_peak_shift_data_CL.dm3"
CL_E, CL_SI,_,_,_ = eels.specload(path+ CL_SIdm3,type='1')

# LL SI
LL_SIdm3 = "160kx_peak_shift_data_LL.dm3"   
LL_E, LL_SI,_,_,_ = eels.specload(path+ LL_SIdm3,type='1')

In [ ]:
## Using the LL_SI, we can generate a shift matrix for each pixel
S,shift_LL=  eels.zlp_shifts(LL_SI,LL_E)

## Using the shift matrix generated above, apply this shift matrix to the core loss SI.
shift_CL_SI = eels.shift_coreloss(CL_SI,S)

## After shifting the spectra, we need to clip the edges of the energy axis and SI to removing artifacts from rolling the array.
clip= int(np.max(S))

eels.easyplot("Carbon edge before and after alignment")
plt.plot(CL_E[clip:-clip],sum(sum(CL_SI))[clip:-clip],label='Unshifted')
plt.plot(CL_E[clip:-clip],sum(sum(shift_CL_SI))[clip:-clip],label='Shifted')
plt.xlim(500,600)
plt.ylim(1e6,4.5e6)
plt.legend()

## As you can see below, this step is crucial in low-mag data. 
## Be careful to check how the spectrum drifts across the SI before running a background subtraction.

## <span style="color:Purple"> 3b. Additional fitting parameters  </span> <a name="fit"/> 

In this section we will breifly go over the many options available in the function ***`eels.bgsub_SI`***. Start by loading in our data from the first notebook.

In [ ]:
# file path for directory with data files 
path = "../Data/T4a/" 

SIdm = "SmBaMnO_onDSO_0p4step_5ms_750eV_0p5disp_5mmap_50pA_SI.dm4"

## Load the SI (spectral image), calibrated energy axis, and the metadata parameters
energy, SI, pxscale, disp, params = eels.specload(path+SIdm)

bin_SI = eels.bin_ndarray(SI[:-1,:,:],bin_factors)

Pick an edge from this SI for these examples.

In [ ]:
eels.fit_check(bin_SI,energy)

 Along with toggling between log and non-log fitting, lba and non-lba fitting, we can also adjust the following:

- **Fit function**: ***`eels.bgsub_SI`***  can handle the following fitting functions for the background - linear, power law, exponential.


In [ ]:
## If you'd like to skip the step of redfining your edge parameters, I've left a few egdes here for you to plug in.
# Sc = [350,396,396,407,'Sc-L2,3']
# Mn = [600,633,636,657,'Mn-L3,2']
# Ba = [708,775,776,807,'Ba-M4,5']
# Sm = [849,1049,1065,1117,'Sm-M4,5']

edge = [??,??,??,??,'label']
edge_ch = []
for element in edge[:-1]:
    edge_ch.append(eels.eVtoCh(element,energy))


bsub_lin = eels.bgsub_SI(bin_SI, energy, edge, gfwhm = 15, log=True, fit='lin')
bsub_exp = eels.bgsub_SI(bin_SI, energy, edge, gfwhm = 15, log=True, fit='exp')
bsub_pl  = eels.bgsub_SI(bin_SI, energy, edge, gfwhm = 15, log=True, fit='pl')

ymax = 1.2*np.max(np.sum(bin_SI,axis=(0,1))[edge_ch[0]:edge_ch[3]])

fig,ax = plt.subplots(1,3,figsize=(10,3))
ax[0].set_title('Linear Fit')
ax[0].plot(energy,np.sum(bin_SI,axis=(0,1)),label='raw data') 
ax[0].plot(energy,np.sum(bin_SI-bsub_lin,axis=(0,1)),label='fit') 
ax[0].plot(energy,np.sum(bsub_lin,axis=(0,1)),label='residual')
ax[0].axhline(y=0,color='k',linestyle='--')
ax[0].set_xlim(edge[0],edge[3]+100)
ax[0].set_ylim(-np.abs(1.2*np.sum(bsub_lin,axis=(0,1))[edge_ch[1]]), ymax)
ax[0].legend()
ax[0].set_yticks([0])

ax[1].set_title('Exponential Fit')
ax[1].plot(energy,np.sum(bin_SI,axis=(0,1)),label='raw data') 
ax[1].plot(energy,np.sum(bin_SI-bsub_exp,axis=(0,1)),label='fit') 
ax[1].plot(energy,np.sum(bsub_exp,axis=(0,1)),label='residual')
ax[1].axhline(y=0,color='k',linestyle='--')
ax[1].set_xlim(edge[0],edge[3]+100)
ax[1].set_ylim(-np.abs(1.2*np.sum(bsub_exp,axis=(0,1))[edge_ch[1]]), ymax)
ax[1].legend()
ax[1].set_yticks([0])

ax[2].set_title('Power Law Fit')
ax[2].plot(energy,np.sum(bin_SI,axis=(0,1)),label='raw data') 
ax[2].plot(energy,np.sum(bin_SI-bsub_pl,axis=(0,1)),label='fit') 
ax[2].plot(energy,np.sum(bsub_pl,axis=(0,1)),label='residual')
ax[2].axhline(y=0,color='k',linestyle='--')
ax[2].set_xlim(edge[0],edge[3]+100)
ax[2].set_ylim(-np.abs(1.2*np.sum(bsub_pl,axis=(0,1))[edge_ch[1]]), ymax)
ax[2].legend()
ax[2].set_yticks([0])

- **Linear combination function**: It is also possible to use a linear combination fit in power law or exponential fittings. When LC is toggled, it will output both the orginial fit and a LC fit corresponding to the fit function chosen.
- **STD**: LC picks two functions for linear combination by fitting a gaussian to the spread of either power laws or exponential values in the SI and picking the values based on the number of standard deviations away from the mean given by 'nstd'.

In [ ]:
bsub_pl,bsub_lcpl= eels.bgsub_SI(bin_SI, energy, edge, fit='pl' ,log=True ,lc=True, gfwhm=gfwhm,nstd=2)

bsub_ex,bsub_lcex= eels.bgsub_SI(bin_SI, energy, edge, fit='exp',log=True ,lc=True, gfwhm=gfwhm,nstd=0.1)


ymax = 1.2*np.max(np.sum(bin_SI,axis=(0,1))[edge_ch[0]:edge_ch[3]])

fig,ax = plt.subplots(2,2,figsize=(8,8))
ax[0][0].set_title('Power Law Fit')
ax[0][0].plot(energy,np.sum(bin_SI,axis=(0,1)),label='raw data') 
ax[0][0].plot(energy,np.sum(bin_SI-bsub_pl,axis=(0,1)),label='fit') 
ax[0][0].plot(energy,np.sum(bsub_pl,axis=(0,1)),label='residual')
ax[0][0].axhline(y=0,color='k',linestyle='--')
ax[0][0].set_xlim(edge[0],edge[3]+100)
ax[0][0].set_ylim(-np.abs(1.2*np.sum(bsub_pl,axis=(0,1))[edge_ch[1]]), ymax)
ax[0][0].legend()
ax[0][0].set_yticks([0])

ax[1][0].set_title('Linear Combination Power Law Fit')
ax[1][0].plot(energy,np.sum(bin_SI,axis=(0,1)),label='raw data') 
ax[1][0].plot(energy,np.sum(bin_SI-bsub_lcpl,axis=(0,1)),label='fit') 
ax[1][0].plot(energy,np.sum(bsub_lcpl,axis=(0,1)),label='residual')
ax[1][0].axhline(y=0,color='k',linestyle='--')
ax[1][0].set_xlim(edge[0],edge[3]+100)
ax[1][0].set_ylim(-np.abs(1.2*np.sum(bsub_lcpl,axis=(0,1))[edge_ch[1]]), ymax)
ax[1][0].legend()
ax[1][0].set_yticks([0])

ax[0][1].set_title('Exponential Fit')
ax[0][1].plot(energy,np.sum(bin_SI,axis=(0,1)),label='raw data') 
ax[0][1].plot(energy,np.sum(bin_SI-bsub_ex,axis=(0,1)),label='fit') 
ax[0][1].plot(energy,np.sum(bsub_ex,axis=(0,1)),label='residual')
ax[0][1].axhline(y=0,color='k',linestyle='--')
ax[0][1].set_xlim(edge[0],edge[3]+100)
ax[0][1].set_ylim(-np.abs(1.2*np.sum(bsub_ex,axis=(0,1))[edge_ch[1]]), ymax)
ax[0][1].legend()
ax[0][1].set_yticks([0])

ax[1][1].set_title('Linear Combination Exponential Fit')
ax[1][1].plot(energy,np.sum(bin_SI,axis=(0,1)),label='raw data') 
ax[1][1].plot(energy,np.sum(bin_SI-bsub_lcex,axis=(0,1)),label='fit') 
ax[1][1].plot(energy,np.sum(bsub_lcex,axis=(0,1)),label='residual')
ax[1][1].axhline(y=0,color='k',linestyle='--')
ax[1][1].set_xlim(edge[0],edge[3]+100)
ax[1][1].set_ylim(-np.abs(1.2*np.sum(bsub_lcex,axis=(0,1))[edge_ch[1]]), ymax)
ax[1][1].legend()
ax[1][1].set_yticks([0])

- **Non Log Fitting Paramers**: When performing a non log fit, it is sometimes necessary to adjust the fitting parameters within the non-linear optimization function. Below is a list of the parameters you can adjust:

Non log fitting parameters: Adjust the strictness of a non-log fit using these parameters:

**ftol** - default to 0.0005, Relative error desired in the sum of squares.

**gtol** - default to 0.00005, Orthogonality desired between the function vector and the columns of the Jacobian.

**xtol** - default to None, Relative error desired in the approximate solution.

**maxfev** - default to 50000, Only change if you are consistenly catching runtime errors and loosening gtol/ftols are not making a good enough fit.

**method** - default is 'trf', see https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html#scipy.optimize.least_squares for description of methods

In [ ]:
bsub_high= eels.bgsub_SI(bin_SI, energy, edge,log=False, gfwhm=gfwhm)

bsub_low = eels.bgsub_SI(bin_SI, energy, edge,log=False, gfwhm=gfwhm,ftol = 1e-1,gtol=1e-1)

ymax = 1.2*np.max(np.sum(bin_SI,axis=(0,1))[edge_ch[0]:edge_ch[3]])

fig,ax = plt.subplots(1,2,figsize=(8,4))
ax[0].set_title('High Tolerance')
ax[0].plot(energy,np.sum(bin_SI,axis=(0,1)),label='raw data') 
ax[0].plot(energy,np.sum(bin_SI-bsub_high,axis=(0,1)),label='fit') 
ax[0].plot(energy,np.sum(bsub_high,axis=(0,1)),label='residual')
ax[0].axhline(y=0,color='k',linestyle='--')
ax[0].set_xlim(edge[0],edge[3]+100)
ax[0].set_ylim(-np.abs(1.2*np.sum(bsub_high,axis=(0,1))[edge_ch[1]]), ymax)
ax[0].legend()
ax[0].set_yticks([0])

ax[1].set_title('Low Tolerance')
ax[1].plot(energy,np.sum(bin_SI,axis=(0,1)),label='raw data') 
ax[1].plot(energy,np.sum(bin_SI-bsub_low,axis=(0,1)),label='fit') 
ax[1].plot(energy,np.sum(bsub_low,axis=(0,1)),label='residual')
ax[1].axhline(y=0,color='k',linestyle='--')
ax[1].set_xlim(edge[0],edge[3]+100)
ax[1].set_ylim(-np.abs(1.2*np.sum(bsub_low,axis=(0,1))[edge_ch[1]]), ymax)
ax[1].legend()
ax[1].set_yticks([0])

# <span style="color:Red"> End of notebook  </span> 
[Return to TOC](#toc)